In [1]:
%matplotlib inline
#%matplotlib notebook
#%matplotlib widget
import matplotlib 
import numpy as np
import pandas as pd
import os, sys, time
#import ipywidgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection    
 
# use LaTeX, choose nice some looking fonts and tweak some settings
matplotlib.rc('font', family='serif')
matplotlib.rc('font', size=16)
matplotlib.rc('legend', fontsize=16)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('legend', handlelength=1.5)
matplotlib.rc('legend', frameon=False)
matplotlib.rc('xtick.major', pad=7)
matplotlib.rc('xtick.minor', pad=7)
matplotlib.rc('text', usetex=True)
# matplotlib.rc('text.latex', 
#               preamble=[r'\usepackage[T1]{fontenc}',
#                         r'\usepackage{amsmath}',
#                         r'\usepackage{txfonts}',
#                         r'\usepackage{textcomp}'])

matplotlib.rc('figure', figsize=(12, 9))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device('cuda:0')

VI. Artificial Neural Network
===

## 머신 러닝 용어 이해하기.


#### 머신 러닝 모델의 평가
실제 모델을 평가하기 위해서 데이터를 훈련용, 검증용, 테스트용 이렇게 세 가지로 분리하는 것이 일반적입니다. 다만, 이 책의 목적은 개념 학습이므로 일부 실습에서는 별도로 세 가지로 분리하지 않고 훈련용, 테스트용으로만 분리해서 사용합니다. 그렇다면 훈련용, 테스트용 두 가지로만 나눠서 테스트 데이터로 한 번만 테스트하면 더 편할텐데 굳이 왜 검증용 데이터를 만들어 놓는 것일까요?

검증용 데이터는 모델의 성능을 평가하기 위한 용도가 아니라, 모델의 성능을 조정하기 위한 용도입니다. 더 정확히는 과적합이 되고 있는지 판단하거나 하이퍼파라미터의 조정을 위한 용도입니다. **하이퍼파라미터(초매개변수)** 란 값에 따라서 모델의 성능에 영향을 주는 매개변수들을 말합니다. 반면, 가중치와 편향과 같은 학습을 통해 바뀌어져가는 변수를 이 책에서는 **매개변수**라고 부릅니다.

이 두 값 하이퍼파라미터와 매개변수의 가장 큰 차이는 <u>하이퍼파라미터는 보통 사용자가 직접 정해줄 수 있는 변수</u>라는 점입니다. 선형 회귀 챕터에서 배우게 되는 경사 하강법에서 학습률(learning rate)이 이에 해당되며 딥 러닝에서는 은닉층의 수, 뉴런의 수, 드롭아웃 비율 등이 이에 해당됩니다. 반면 <u>매개변수는 사용자가 결정해주는 값이 아니라 모델이 학습하는 과정에서 얻어지는 값</u>입니다. 정리하면 하이퍼파라미터는 사람이 정하는 변수인 반면, 매개변수는 기계가 훈련을 통해서 바꾸는 변수라고 할 수 있으며 이 책에서는 이와 같은 기준으로 변수의 이름을 명명합니다.

훈련용 데이터로 훈련을 모두 시킨 모델은 검증용 데이터를 사용하여 정확도를 검증하며 하이퍼파라미터를 튜닝(tuning)합니다. 또한 이 모델의 매개변수는 검증용 데이터로 정확도가 검증되는 과정에서 점차 검증용 데이터에 점점 맞추어져 가기 시작합니다.

#### 분류(Classification) 와 회귀(Regression)

